## 0. 준비 

In [1]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#라이브러리 import
import pandas as pd
import numpy as np
import os
import moviepy.editor
import requests
import json
from moviepy.editor import VideoFileClip, concatenate_videoclips

경로 설정

In [8]:
#폴더 생성
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [9]:
movie_name="midnightsun" # 영화 제목 입력
mydrive="/content/drive/MyDrive/gp/"
createFolder(mydrive+movie_name) # 영화 제목 폴더 생성

In [5]:
#비디오 클립 경로 지정
createFolder(mydrive+movie_name+"/clip") 
video_path= mydrive+movie_name+"/"+movie_name+".mp4"

## 1. STT API

In [10]:
#비디오를 오디오로 바꾸는 과정
clip = VideoFileClip(video_path)
audio=clip.audio.write_audiofile(mydrive+movie_name+"/audio.mp3")

[MoviePy] Writing audio in /content/drive/MyDrive/gp/midnightsun/audio.mp3


100%|██████████| 18724/18724 [00:30<00:00, 609.88it/s]

[MoviePy] Done.


In [ ]:
class ClovaSpeechClient:
    # Clova Speech invoke URL
    invoke_url = 'https://clovaspeech-gw.ncloud.com/external/v1/2683/aabd54b3f93690a9bb802485794bd320b3dae7d5563123d5b2fcef424bf434ca'
    # Clova Speech secret key
    secret = '9ec2f99f976a40c3bbaf22df24474ef7'

    def req_url(self, url, completion, callback=None, userdata=None, forbiddens=None, boostings=None, wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'url': url,
            'language': 'en-US',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'Content-Type': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        return requests.post(headers=headers,
                             url=self.invoke_url + '/recognizer/url',
                             data=json.dumps(request_body).encode('UTF-8'))

    def req_object_storage(self, data_key, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                           wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'dataKey': data_key,
            'language': 'en-US',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'Content-Type': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        return requests.post(headers=headers,
                             url=self.invoke_url + '/recognizer/object-storage',
                             data=json.dumps(request_body).encode('UTF-8'))

    def req_upload(self, file, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                   wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'language': 'en-US',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        print(json.dumps(request_body, ensure_ascii=False).encode('UTF-8'))
        files = {
            'media': open(file, 'rb'),
            'params': (None, json.dumps(request_body, ensure_ascii=False).encode('UTF-8'), 'application/json')
        }
        response = requests.post(headers=headers, url=self.invoke_url + '/recognizer/upload', files=files)
        return response

if __name__ == '__main__':
    # res = ClovaSpeechClient().req_url(url='http://example.com/media.mp3', completion='sync')
    # res = ClovaSpeechClient().req_object_storage(data_key='data/media.mp3', completion='sync')
    res = ClovaSpeechClient().req_upload(file=audio, completion='sync')
    print(res.text)


##2. 데이터 전처리

In [11]:
#데이터 로드
file_path= mydrive+movie_name+"/sttout"
df = pd.read_csv(file_path+'.csv')
df.head()

,Index,Modification,Speaker,Start Time,End Time,Script
0,1,인식결과,1,2830,10490,i have this reoccurring dream i'm a little gir...
1,1,수정결과,A,2830,10490,i have this reoccurring dream i'm a little gir...
2,2,인식결과,1,11380,19490,i feel the warmth of the sun on my skin and th...
3,2,수정결과,A,11380,19490,i feel the warmth of the sun on my skin and th...
4,3,인식결과,1,20730,29690,forever when i wake up i missed the dream i wa...


1. 인식결과 행만 남김 (수행결과 행 삭제)

In [12]:
df=df.assign(g=df.index % 2).query('g==0') # 짝수행만 남김
df.head() 

,Index,Modification,Speaker,Start Time,End Time,Script,g
0,1,인식결과,1,2830,10490,i have this reoccurring dream i'm a little gir...,0
2,2,인식결과,1,11380,19490,i feel the warmth of the sun on my skin and th...,0
4,3,인식결과,1,20730,29690,forever when i wake up i missed the dream i wa...,0
6,4,인식결과,1,29830,38440,that would be impossible because my mom died w...,0
8,5,인식결과,1,39080,48190,i have a rare genetic condition called zero de...,0


2. 필요 없는 colums 삭제

In [13]:
df=df.drop(columns='g')  #g행 삭제
df=df.drop('Speaker',axis='columns') # speaker 열 삭제
df= df.drop('Modification',axis='columns') #modification 열 삭제
df= df.drop('Index',axis='columns') #modification 열 삭제

3. Script에 결측치 있는 colums삭제

In [14]:
df=df.dropna(axis='index',how='any')

4.  인덱스 초기화

In [15]:
df= df.reset_index(drop=True)

In [16]:
df.head() 

,Start Time,End Time,Script
0,2830,10490,i have this reoccurring dream i'm a little gir...
1,11380,19490,i feel the warmth of the sun on my skin and th...
2,20730,29690,forever when i wake up i missed the dream i wa...
3,29830,38440,that would be impossible because my mom died w...
4,39080,48190,i have a rare genetic condition called zero de...


## 3.New Time Stemp 

In [17]:
df['start'] = df.apply(lambda row : (row['Start Time']+ row['End Time'])/2-5000, axis = 1)
df['end'] = df.apply(lambda row : (row['Start Time']+ row['End Time'])/2+5000, axis = 1)

df=df.drop('Start Time',axis='columns')
df=df.drop('End Time',axis='columns')

df.head() 

,Script,start,end
0,i have this reoccurring dream i'm a little gir...,1660.0,11660.0
1,i feel the warmth of the sun on my skin and th...,10435.0,20435.0
2,forever when i wake up i missed the dream i wa...,20210.0,30210.0
3,that would be impossible because my mom died w...,29135.0,39135.0
4,i have a rare genetic condition called zero de...,38635.0,48635.0


## 4.text(대사) list 제작 

In [18]:
df_temp=df #임의 배열 만듬
df_temp=df_temp.drop('start',axis='columns') 
df_temp=df_temp.drop('end',axis='columns')

In [19]:
df_temp.to_csv(file_path+'_text_list.csv',index=True) #csv파일로 저장

## 5.비디오 clip 제작

In [23]:
#clip 저장 코드
n=0
for start, end in zip(df['start'], df['end']):
  clip = VideoFileClip(video_path).subclip(start*0.001,end*0.001)
  clip.write_videofile(mydrive+movie_name+"/clip/"+str(n)+".mp4")
  print(str(n)+"번째 클립 저장 완료")
  n=n+1

[MoviePy] >>>> Building video /content/drive/MyDrive/gp/midnightsun/clip/0.mp4
[MoviePy] Writing audio in 0TEMP_MPY_wvf_snd.mp3


100%|██████████| 221/221 [00:00<00:00, 685.55it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/drive/MyDrive/gp/midnightsun/clip/0.mp4



100%|█████████▉| 300/301 [00:23<00:00, 12.50it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/gp/midnightsun/clip/0.mp4 

0번째 클립 저장 완료
[MoviePy] >>>> Building video /content/drive/MyDrive/gp/midnightsun/clip/1.mp4
[MoviePy] Writing audio in 1TEMP_MPY_wvf_snd.mp3


100%|██████████| 221/221 [00:00<00:00, 635.65it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/drive/MyDrive/gp/midnightsun/clip/1.mp4



 19%|█▉        | 57/300 [00:02<00:09, 26.90it/s]


KeyboardInterrupt: ignored